In [1]:
import numpy as np
import pandas as pd
import pypianoroll as pr
import matplotlib.pyplot as plt
import importlib
import json
import xml.etree.ElementTree as ET
import os
import sys
import pickle
from time import time

import torch
from torch import nn
import numpy as np
from torch.nn import functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable

#### 读取数据

In [2]:
sys.path.append('..')
import dataloader
#importlib.reload(dataloader)
import model
#importlib.reload(model)
from dataloader import DataLoader
#importlib.reload(DataLoader)
from model.DeepModel import * 
#importlib.reload(model.DeepModel)
DL = DataLoader.DataLoader(device='cpu')
# DL.process_raw_data()

reading chord_num_dic
reading num_chord_dic
reading double_compressed_data


#### 统计所有和弦数目，出现的三和弦数目以及全部使用三和弦的样本数

In [3]:
total = 0  # 出现的和弦总数
triad_num = 0  # 三和弦总数
triad_num_dic = {i:0 for i in DL.chord_num_dic.keys()}
triad_sample_num = 0 # 全部使用三和弦的样本数
triad_samples = []
for sample in DL.double_compressed_data:
    sample_copy = sample.copy()
    melody = sample['melody']
    chord = sample['chord']
    t = True
    for i in range(len(melody)):
        total += 1
        try:
            DL.chord_num_dic[chord[i]]
            triad_num_dic[chord[i]] += 1
            triad_num += 1
        except:
            if len(chord[i]) < 3:
                for c in DL.chord_num_dic.keys():
                    if melody[i] not in c:
                        continue
                    for k in chord[i]:
                        if k not in c:
                            break
                    else:
                        sample_copy['chord'][i] = c
                        triad_num_dic[c] += 1
                        triad_num += 1
                        break
                else:
                    t = False
            else:
                t = False
                pass
    if t:
        triad_samples.append(sample_copy)
        triad_sample_num += 1

print(total)
print(triad_num)
print(triad_sample_num)

306469
220889
5155


In [4]:
DL.get_train_data(min_length=1, write_cache=False, valid_compressed_data=triad_samples)

10000 valid train_data
20000 valid train_data
30000 valid train_data
40000 valid train_data
50000 valid train_data
60000 valid train_data
total 61380 valid train_data


In [7]:
device = 'cuda'
model = ChordLstmNet(chord_num=39, melody_keys=12, hidden_size=128, num_layers=2, bidirectional=False,
                 alpha=0.2, device=device).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-3)

In [ ]:
train_loss = []
test_loss = []
train_data = DL.train_data[:40000]
test_data = DL.train_data[40000:]
train_data = [(sample[0].unsqueeze(0).to(device), sample[1].unsqueeze(0).to(device),
                       sample[2].unsqueeze(0).to(device), sample[3].unsqueeze(0).to(device))
                      for sample in train_data]
test_data = [(sample[0].unsqueeze(0).to(device), sample[1].unsqueeze(0).to(device),
              sample[2].unsqueeze(0).to(device), sample[3].unsqueeze(0).to(device))
             for sample in test_data]
epochs = 40
batch_size = 1000
for epoch in range(epochs):
    num = 0 
    loss = 0
    t = time()
    model.train()
    for sample in train_data:
        if num < batch_size:
            a = sample[0]
            b = sample[1]
            c = sample[2]
            out = model(a,b,c)
            loss += F.nll_loss(out, sample[3])
            num += 1
        else:
            num = 0
            loss /= batch_size
            print('loss: {:.4f} time used: {:.4f}s'.format(loss, time()-t))
            train_loss.append(float(loss.detach().to('cpu')))
            loss.backward()
            loss = 0
            optimizer.step()
            optimizer.zero_grad()
            t = time()
    print('epoch {} testing'.format(epoch))
    loss = 0
    model.eval()
    for sample in test_data:

        a = sample[0]
        b = sample[1]
        c = sample[2]
        out = model(a,b,c)
        loss += F.nll_loss(out, sample[3])

    loss /= len(test_data)
    test_loss.append(float(loss.detach().to('cpu')))
    print('test loss: {:.4f} time used: {:.4f}s'.format(loss, time()-t))

loss: 3.6784 time used: 0.7697s
loss: 3.6701 time used: 0.5425s
loss: 3.6485 time used: 0.6756s
loss: 3.6492 time used: 0.5995s
loss: 3.6226 time used: 0.5855s
loss: 3.6235 time used: 0.5605s
loss: 3.5842 time used: 0.5742s
loss: 3.5737 time used: 0.5355s
loss: 3.5486 time used: 0.6086s
loss: 3.5285 time used: 0.6436s
loss: 3.4826 time used: 0.5915s
loss: 3.4795 time used: 0.5785s
loss: 3.4318 time used: 0.5895s
loss: 3.3484 time used: 0.5431s
loss: 3.2739 time used: 0.5715s
loss: 3.1333 time used: 0.5985s
loss: 3.0169 time used: 0.6256s
loss: 3.0339 time used: 0.5585s
loss: 2.9647 time used: 0.5987s
loss: 2.6870 time used: 0.5605s
loss: 2.6406 time used: 0.5681s
loss: 2.9883 time used: 0.5635s
loss: 2.9784 time used: 0.5725s
loss: 2.8962 time used: 0.6716s
loss: 2.6297 time used: 0.5815s
loss: 2.8368 time used: 0.5755s
loss: 2.6788 time used: 0.6146s
loss: 2.7337 time used: 0.5911s
loss: 2.6071 time used: 0.6326s
loss: 2.6966 time used: 0.5795s
loss: 2.5467 time used: 0.5865s
loss: 2.

In [32]:
float(loss.detach().to('cpu'))

974.814208984375